## Descrição do Notebook

Neste notebook, os dados foram agregados por semana e as regras aplicadas estão descritas ao decorrer do código.

In [1]:
import pandas as pd 
import numpy as np
import xlrd
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv("../datasets/df_rpo_v3.csv")

df['DATA_TRAT.'] = pd.to_datetime(df["DATA_TRAT."], dayfirst = True)


Retirando P10 e P90 das vendas dos produtos:

In [3]:
df.head()

,SAP+NOME,UF,DATA_TRAT.,Vendas Reais Qtde,Vendas Reais $,CMV,Margem Líquida Real %,Margem Líquida Real $,Venda Líquida Real $,Preço,preço2,CMV/QTD,Mg %
0,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-12,1.0,4.99,1.75,52.445652,1.93,3.68,4.99,4.99,1.75,0.524457
1,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-15,1.0,4.99,1.81,50.815217,1.87,3.68,4.99,4.99,1.81,0.508152
2,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-16,1.0,4.99,1.81,50.815217,1.87,3.68,4.99,4.99,1.81,0.508152
3,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-22,2.0,9.98,3.62,50.815217,3.74,7.36,4.99,4.99,1.81,0.508152
4,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-02-12,1.0,2.99,1.81,17.727273,0.39,2.20,2.99,2.99,1.81,0.177273


Criando o *Dataframe* agregado por semana

In [4]:
# Criando colunas de Semana e Ano para auxiliar na agregação
df.reset_index(inplace = True, drop = True)
df["Semana"] = [x.isocalendar()[1] for x in df["DATA_TRAT."]]
df["Ano"] = df["DATA_TRAT."].dt.year

> Fazendo as agregações: 
* Colunas de **Vendas Reais, Margem e CMV** foram somadas
* Colunas de **Preço  e Porcentagem de Margem** foram calculadas com base na média
* Coluna da razão **CMV/QTD** foi retirada uma nova fração entre a coluna CMV e QTD para respeitar a razão entre as grandezas.

In [5]:
df_agg = df[["SAP+NOME", "UF", "Ano", "Semana",  "preço2", "Mg %",
                     "Margem Líquida Real %", "Preço"]].groupby(["SAP+NOME", "UF", "Semana", "Ano"]).mean().reset_index()

df_agg_sum = df[["SAP+NOME", "UF", "Ano", "Semana", "Vendas Reais Qtde", "Vendas Reais $",
                     "Margem Líquida Real $", "Venda Líquida Real $", "CMV"]]. \
             groupby(["SAP+NOME", "UF", "Semana", "Ano"]).sum().reset_index()
df_agg_sum["CMV/QTD"] = df_agg_sum["CMV"] / df_agg_sum["Vendas Reais Qtde"]

Unindo os dois *dataframes* de agregação criados para chegar ao *dataset* que será depositado no *Data Lake*

In [6]:
df_final = df_agg.copy()
df_final = df_final.merge(df_agg_sum, on = ["SAP+NOME", "UF", "Ano", "Semana"], how = "inner")
df_final.reset_index(inplace = True, drop = True)
df_final

,SAP+NOME,UF,Semana,Ano,preço2,Mg %,Margem Líquida Real %,Preço,Vendas Reais Qtde,Vendas Reais $,Margem Líquida Real $,Venda Líquida Real $,CMV,CMV/QTD
0,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2,2017,4.99,0.516304,51.630435,4.990000,2.0,9.980000,3.800000,7.36,3.56000,1.780000
1,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2,2018,4.99,0.524457,52.445652,4.990000,1.0,4.990000,1.930000,3.68,1.75000,1.750000
2,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2,2019,4.99,0.550679,55.067935,5.239500,1.0,5.239500,2.026500,3.68,1.83750,1.837500
3,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,3,2017,4.99,0.508152,50.815217,4.990000,3.0,14.970000,5.610000,11.04,5.43000,1.810000
4,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,3,2018,4.99,0.508152,50.815217,4.990000,2.0,9.980000,3.740000,7.36,3.62000,1.810000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35406,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,51,2018,4.99,0.300713,30.071307,4.796444,111.0,528.975800,114.372000,383.97,271.11160,2.442447
35407,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,51,2019,4.99,0.327262,32.726199,5.067237,83.0,418.494090,94.651100,292.82,210.43418,2.535352
35408,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,52,2017,4.99,0.320648,32.064832,4.915119,179.0,881.220000,206.790000,642.19,435.40000,2.432402
35409,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,52,2018,4.99,0.324613,32.461349,4.952597,186.0,920.327500,216.543500,668.11,454.25460,2.442229


Retirando *P10* e *P90* dos produtos que possuiram variações de preço mais de 10 vezes. Essa agregação foi feita por **Produto** e por **UF** para eliminar os preços promocionais e as sazonalidades de vendas causadas por eles. E para as demais situações de um dado **Produto** ou **UF**, onde o preço não variou mais de 10 vezes, mas, ainda sim, há sazonalidade na quantidade de vendas semanais, também foram removidos os valores menores que *P10*, para remover quantidades insignificantes e *P90*, para remover valores que destoam.

In [7]:
lst_uf = df_final["UF"].unique()
lst_sap = df_final["SAP+NOME"].unique()

for uf in lst_uf:
    for sap in lst_sap:
        df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
        if df_seg.empty == True:
               continue
        else:
            # Se preço variar mais de 10 vezes, encontraremos o p10 e o p90 de valor e quantidade
            if len(df_seg["Preço"].unique()) > 10:
                q10_p = np.quantile(df_seg["Preço"].unique(), .10)
                q90_p = np.quantile(df_seg["Preço"].unique(), .90)
                q_10_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .10)
                q_90_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .90)
                
                # Atribuindo a 'ind', todos os valores que atendem as condições
                ind = df_seg.loc[((df_seg["Preço"] < q10_p)  | (df_seg["Preço"] > q90_p)) &
                                 ((df_seg["Vendas Reais Qtde"] < q_10_qtd) | (df_seg["Vendas Reais Qtde"] > q_90_qtd))]
                
                # Dropando index dos valores de 'ind'
                df_final = df_final.drop(ind.index)
            
            # Atribuindo a df_seg o valor de df_final ao final da condição
            df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
            
            # Pegando o p10 e o p90 da coluna Vendas Reais Qtde
            p10 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .10)
            p90 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .90)
            
            # Atribuindo a variável ind todos os index que são menores que o p10 e maiores que o p90
            ind = df_seg.loc[(df_seg["Vendas Reais Qtde"] < p10)  | (df_seg["Vendas Reais Qtde"] > p90)]
            
            # Removendo do DF todos os index que foram atribuidos a variável ind
            df_final = df_final.drop(ind.index)

In [8]:
df_final = df_final.sort_values(by = ["Ano", "Semana", "UF"]).reset_index(drop = True)

In [9]:
df_final.to_csv("../datasets/df_rpo_v4.csv", index = False)